### (0) IMPORT

In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#BORROWING KAISHENGS CLIENT SECRET AND ID THANKS KS BB PLS DONT DISABLE YA
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

### (1) Playlist extractor
Make sure you do not already have a file called "User_Playlist.csv". if you do, please delete it if not this code will encounter an error

In [2]:
import time

# def extract_playlistID(playlists,find_playlist):
#     while True:
#         for playlist in playlists['items']:
#         ##Loop and return playlistID if found
#             if playlist["name"] == find_playlist:
#                 return playlist['id']
#         ##Show next 50 songs
#         if playlists['next']:
#             playlists = sp.next(playlists)
#         else:
#             return None
        
def extract_trackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_nexttrackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_songInfo(song):
    info = sp.track(song)
    ##Define features u wanna extract here
    id = song
    songName = info["name"]
    albumName = info["album"]["name"]
    albumID = info["album"]["id"]
    artists = info["artists"][0]["name"]
    for artist in info["artists"]:
        artist_genre = sp.artist(artist["id"])
        genre = artist_genre["genres"]
    explicit = info["explicit"]
    releaseDate = info["album"]["release_date"]
    return [id,songName,albumName,albumID,artists,genre,explicit,releaseDate]

def extract_features(song):
    info = sp.audio_features(song)[0]
    ##Define features u wanna extract here
    danceability = info["danceability"]
    energy = info["energy"]
    key = info["key"]
    loudness = info["loudness"]
    mode = info["mode"]
    speechiness = info["speechiness"]
    acousticness = info["acousticness"]
    instrumental = info["instrumentalness"]
    liveness = info["liveness"]
    valence = info["valence"]
    tempo = info["tempo"]
    duration_ms = info["duration_ms"]
    time_signature = info["time_signature"]
    return [danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,
            valence,tempo,duration_ms,time_signature]

#Auth Stuff
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)
    
playlists_name = []
playlists_ID = []

## Ask to insert playlist ID
while True:
    url = input("Insert Spotify Playlist ID (Else type 'done'): ")
    if url == "done":
        break
    else:
        playlists_ID.append(url)
    
# Get Playlist Name
print("Obtaining the Following Playlists: ")
for i in playlists_ID:
    playlist_info = sp.playlist(i)
    name = playlist_info["name"]
    print(name)
    playlists_name.append(name)

##Get Info in Playlist with PlaylistID 
tracksID = {}
for i in range(len(playlists_ID)):
    track_list = []
    playlist_tracks = sp.playlist(playlists_ID[i])
    playlist_tracks = playlist_tracks["tracks"]
    track_list = extract_trackID(playlist_tracks)
    for page in range(0,2):
        if playlist_tracks['next']:
            print("More than 100 Songs Detected. WAITT")
            time.sleep(30)
            print("Looking at next 100 songs")
            playlist_tracks = sp.next(playlist_tracks)
            track_list += extract_nexttrackID(playlist_tracks)
    tracksID[playlists_name[i]] = track_list


##Get Other Track Details
results_tocsv = []
results_tocsv.append(["id","songName","albumName","albumID","artist","genre","explicit","releaseDate",
                     "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                    "valence","tempo","duration_ms","time_signature"])
for i in playlists_name:  
    count = 0
    for song in tracksID[i]:
        time.sleep(1)
        song_info = extract_songInfo(song)
        ##Get Audio Features
        song_features = extract_features(song)
        song_info.extend(song_features)
        results_tocsv.append(song_info)
        count += 1
        
        if count == 100:
            print("Trying not to let spotify kill me. 30s wait :D")
            time.sleep(30)
            count = 0

##Use pandas to convert to dataframe
df = pd.DataFrame(results_tocsv[1:], columns = results_tocsv[0])

##Export to CSV
file_name = "User_Playlist.csv"
df.to_csv(file_name, encoding='utf_8_sig')
print("20s Cooldown...")
time.sleep(20)

NameError: name 'SpotifyClientCredentials' is not defined

### (2) FUNCTIONSSS

In [1]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)  
    return tf_df

def get_features_database(databaseDF):
    #Select Features
    databaseDF = databaseDF[["id","songName",
                "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                "valence","tempo","type"]]

    #OHE Features
    key_ohe = ohe_prep(databaseDF, 'key','key') * 0.5
    mode_ohe = ohe_prep(databaseDF, 'mode','mode') * 0.5

    ##Normalise/Scale Audio Columns
    float_cols = databaseDF.dtypes[databaseDF.dtypes == 'float64'].index.values
    floats = databaseDF[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    ##Combine all Features
    final = pd.concat([floats_scaled, key_ohe, mode_ohe, databaseDF["type"]], axis = 1)
    return final

### (3) GENERALISING/SIMPLIFYING Genre Name of Input Songs to Match Kaggle DS for recommendation
Reduces the highly specific genres of Input Songs retrieved spotipy to increase match with kaggle ds with generalised genres (pre cleaned, retrieved from web)
If METHOD1 does not work for particular song, do METHOD2 so on and so forth <br><br>
METHOD1 - Find User Songs in Dataset and Copy Over <br>
METHOD2 - Webscrap from get_genre.com which genre name is generic and similar to dataset<br>


### (3.1) Importing Web driver and BeautifulSoup

METHOD2 - Webscraping Function: REQUIRES SELENIUM AND BS4 LIBRARY. FOR SELENIUM NEEDS TO INSTALL CHROMEDRIVERS TOO from below: <br>


[CHROMEDRIVERS](https://googlechromelabs.github.io/chrome-for-testing) 

In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

def scrap_genre(artistName):

    ##URL of website to scrap
    url = f"https://www.getgenre.com/artist/{artistName}"
    ##CHANGE PATH TO PATH OF DOWNLOADED CHROMEDRIVER
    driver = webdriver.Chrome(executable_path="/Users/kaisheng/Documents/chromedriver")

    # WAIT FOR BUTTON TAG TO LOAD BEFORE CONTINUING
    driver.implicitly_wait(30)
    driver.get(url)

    classtext = "MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium css-x3ahaf"
    button = driver.find_element_by_xpath(f"//button[contains(@class, '{classtext}')]")
    time.sleep(2)
    # Parse the HTML content of the button with BS4
    html = button.get_attribute("outerHTML")
    driver.close()
    driver.quit()

    #Use BS4 to read HTML segement and extract Genre Text
    soup = BeautifulSoup(html, "html.parser")
    try:
        # Find html button tag with class to obtained genre text
        genre = soup.find('button', class_=classtext).get_text()
        return genre.lower()
    except:
        pass


In [7]:
##Import User Playlist
userDF = pd.read_csv("User_Playlist.csv")
userDF = userDF.drop(columns=["Unnamed: 0"])
##Import Dataset
df = pd.read_csv("spotify_data_cleaned.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna()

#Extract all genres from dataset
datasetGenres_list = []
for i in df["genre"]:
    if i not in datasetGenres_list:
        datasetGenres_list.append(i)

# Flag to indicate whether need to use Method 3 or not
useSpotifyGenre = False

song_genres_list = []
##Find Playlist Songs in Database and replace genre

for index, row in userDF.iterrows():
    id = row["id"]
    artistName = row["artist"]

    ##List to include all genres from one song
    one_song_genre = []

    ##Variable to check if song in database
    dataset_genre = df[df["id"] == id]

    if not dataset_genre.empty:
        ##METHOD1 - Searches for kaggle ds for song inputted to retrieve genre (matching kaggle ds). 
        ##If found, genre from kaggle ds replaces highly specific genre of song retrieved from spotify
        one_song_genre = dataset_genre["genre"].to_list()
    else:
        ##METHOD2 - Web Scraping to retrieve simplified genre from getgenre.com to replace highly specific genre retrieved from spotify
        genre = scrap_genre(artistName)
        if genre != "":
            one_song_genre.append(genre)

    ##CHECKS IF METHOD 1 or 2 managed to obtain generalised genre data
    song_genres_list.append(one_song_genre)

userDF["genre"] = song_genres_list
userDF

,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,37BH3RrKhRaHMQwuhlkEaF,Bummed,Bummed,38AOH6bBpJDYTn1EAaCMSn,Chet Porter,[electropop],True,2020-03-05,0.437,0.759,...,-6.133,1,0.039,0.000371,0,0.0872,0.382,160.034,184800,4


### (4) Find Top 3 Genres in User Playlist By Counting Frequency of Genres

In [8]:
# Find top 3 genres in playlist
exploded_df = userDF.explode('genre')

freq = exploded_df["genre"].value_counts(sort=True)

genres_count = dict(freq.head(3))
genre_top3 = list(genres_count.keys())
        
print("Top 3 Genre's of User Playlist:" + str(genre_top3))
exploded_df

Top 3 Genre's of User Playlist:['electropop']


,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,37BH3RrKhRaHMQwuhlkEaF,Bummed,Bummed,38AOH6bBpJDYTn1EAaCMSn,Chet Porter,electropop,True,2020-03-05,0.437,0.759,...,-6.133,1,0.039,0.000371,0,0.0872,0.382,160.034,184800,4


### (5) Extract Both User and Dataset Csv

In [9]:
#Extract Database CSV
databaseDF = pd.read_csv("spotify_data_cleaned.csv", encoding="utf_8_sig")
databaseDF = databaseDF.drop(columns=['Unnamed: 0'])
#Selects columns that we want
databaseDF = databaseDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]]

#Create new feature/column of artist_songName
databaseDF["artist_songName"] = databaseDF["artist"] + "_" + databaseDF["songName"]

##Change all values to lowercase
databaseDF["artist_songName"] = databaseDF["artist_songName"].str.lower()

#Check for duplicates in database
databaseDF = databaseDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

In [10]:
#Extract User Playlist CSV
playlistDF = pd.read_csv("User_Playlist.csv", encoding="utf_8_sig")

playlistDF = playlistDF.drop(columns=['Unnamed: 0'])
#Reset Index
playlistDF = playlistDF.reset_index(drop = True)  
#Selects columns that we want
playlistDF = playlistDF[["id","artist","songName","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]] 

#Create new feature/column of artist_songName to remove duplicate later on
playlistDF["artist_songName"] = playlistDF["artist"] + "_" + playlistDF["songName"]
##Change all values to lowercase
playlistDF["artist_songName"] = playlistDF["artist_songName"].str.lower()

#Check for duplicates in user Playlist
playlistDF = playlistDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

### (6) Normalise Features

In [11]:
#Merge user + dataset dataframe to normalise 
#Normalise takes min and max in dataframe as reference and change it to 0 and 1 respectively

#Group the dataframe as we gonna split it again alter
databaseDF["type"] = "Dataset"
playlistDF["type"] = "User"


#Merge the 2 datasets together
combinedDF = pd.concat([databaseDF,playlistDF], ignore_index=True)

#Check for duplicates between user and Database
combinedDF = combinedDF.drop_duplicates(subset=["artist_songName"], keep="last",ignore_index= True)

##Update databaseDF with removed songs from user Playlist
databaseDF = combinedDF[(combinedDF["type"]== "Dataset")]

##Normalise and get Vectors for Dataset + User
normalised_vector = get_features_database(combinedDF)

##Seperate User from databaseDF 
database_vector = normalised_vector[normalised_vector["type"] == "Dataset"]
user_vector = normalised_vector[normalised_vector["type"] == "User"]

#Drop "type" column
database_vector = database_vector.drop(columns="type")
user_vector = user_vector.drop(columns="type")
databaseDF = databaseDF.drop(columns=["type"])


### (7) Merge all User Vector to form single vector (this is where we gotta do sth w clustering model to not squeeze every song vector into one ball of mess bc the recos are so shit)

In [12]:
##Single Vector Creation
final_user_vector_list = []
for i in user_vector.columns:
    final_user_vector_list.append(user_vector[i].sum()/len(user_vector[i]))

#Putting into a vector dataframe
final_user_vector = pd.DataFrame(columns=user_vector.columns,)
final_user_vector.loc[0] = final_user_vector_list

final_user_vector

,danceability,energy,loudness,speechiness,acousticness,instrumental,liveness,valence,tempo,key|0,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,0.088105,0.1518,0.160479,0.008033,0.000074,0.0,0.01744,0.0764,0.128079,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5


### (LAST) RECOMMEND TIME!
if the database songs that have high cosine similarity as input song DO NOT SHARE SAME GENRE (ALL FROM SPOTIFY BTW), THEN WE WILL GET NO RECOS REGARDLESS

SO if the results is empty, it simply means the kaggle ds does not contain songs of that genre

In [16]:
def generate_rec(databaseDF, database_vector, user_vector, genre_top3):
    #Cosine Similarity
    databaseDF["sim"] = cosine_similarity(database_vector,user_vector)
    # Drop rows with different genre from top 3 genres
    if len(genre_top3) == 1:
        datasetChunk = databaseDF[(databaseDF["genre"] == genre_top3[0])]
    elif len(genre_top3) == 2:
        datasetChunk = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1])]
    else:
        datasetChunk = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1]) | (databaseDF["genre"] == genre_top3[2])]
    
    ##Check if user genre is in database
    if datasetChunk.empty:
        return databaseDF.sort_values('sim',ascending = False).head()
    
    rec_sorted = datasetChunk.sort_values('sim',ascending = False)
    rec_top5 = pd.DataFrame(columns=rec_sorted.columns)

    ##Web Scrap Dataset
    for index, row in rec_sorted.iterrows():    
        artist = row["artist"]
        genre = scrap_genre(artist)
        rec_top5 = rec_top5.dropna(axis=1, how='all')
        if genre != "" and genre in genre_top3: 
            rec_top5 = pd.concat([rec_top5, pd.DataFrame([row])], ignore_index=True)
        if len(rec_top5) == 5:
            return rec_top5
        
            
    # #Sort and recommend top 5 with same genres
    # rec_top5 = databaseDF.sort_values('sim',ascending = False).head()
    # return rec_top5

In [17]:
##Generate Recc Songs
result = generate_rec(databaseDF,database_vector,final_user_vector, genre_top3)
result

,id,artist,songName,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,artist_songName,sim
426147,3PC2spBvaHmQzlQF2uw95N,Tyler Farr,Soundtrack to a Small Town Sundown,country,0.435,0.765,8,-4.699,1,0.0388,0.006110,0.000000,0.1030,0.342,149.944,tyler farr_soundtrack to a small town sundown,0.999859
883774,1NyCFoQbnTgpJpUfsDfaLC,Tokio Hotel,Jung Und Nicht Mehr Jugendfrei,german,0.448,0.818,8,-4.735,1,0.0460,0.003140,0.000000,0.0967,0.372,149.912,tokio hotel_jung und nicht mehr jugendfrei,0.999798
239237,41imURTSBkObv3x3azvkeB,Avi8,Carry Me,hardstyle,0.480,0.807,8,-6.418,1,0.0463,0.000387,0.025900,0.1180,0.391,155.006,avi8_carry me,0.999787
969932,6H0Sx0sukJPoYdc8xRenLa,Motion City Soundtrack,Not Asking You To Leave,emo,0.420,0.825,8,-5.008,1,0.0498,0.000244,0.000000,0.0834,0.345,159.979,motion city soundtrack_not asking you to leave,0.999782
72874,3G10QUHtxEsMGcZYgvdauS,Modern Chemistry,Never Scared,emo,0.471,0.762,8,-4.630,1,0.0552,0.002900,0.000379,0.1080,0.334,147.917,modern chemistry_never scared,0.999753
